In [ ]:
%pip install -I scikit-learn==0.24.1

     |████████████████████████████████| 22.3MB 127kB/s 
     |████████████████████████████████| 28.5MB 106kB/s 
     |████████████████████████████████| 15.7MB 198kB/s 
     |████████████████████████████████| 307kB 39.9MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
%matplotlib notebook
import plotly.express as px
import numpy as np
# import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.base import TransformerMixin
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.kernel_approximation import Nystroem
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_curve, auc
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import label_binarize
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering, AgglomerativeClustering
from scipy.stats import mannwhitneyu, chisquare

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df_crime = pd.read_csv('gdrive/MyDrive/Colab Notebooks/chicago_sampled_data.csv', index_col=0)
df_crime['Date'] = df_crime['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ))
df_crime

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11645833,JC213044,2012-05-05 12:25:00,057XX W OHIO ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,1511,15.0,29.0,25.0,11,NaN,NaN,2012,04/06/2019 04:04:43 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11227247,JB147078,2012-01-01 09:00:00,105XX S INDIANAPOLIS AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,432,4.0,10.0,52.0,11,NaN,NaN,2012,02/11/2018 03:57:41 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10225605,HY412867,2012-07-11 09:00:00,017XX W ALBION AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,2432,24.0,40.0,1.0,11,1163498.0,1943889.0,2012,02/10/2018 03:50:01 PM,42.001670,-87.673864,"(42.00167049, -87.673863642)",46.0,21853.0,10.0,242.0,5.0,9.0,11.0,41.0
10225648,HY412901,2011-08-01 08:00:00,080XX S TRUMBULL AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,834,8.0,18.0,70.0,11,1154779.0,1851149.0,2011,02/09/2018 03:44:29 PM,41.747362,-87.708424,"(41.747362057, -87.708423712)",6.0,4300.0,69.0,199.0,30.0,8.0,13.0,232.0
11042125,JA376558,2011-12-16 00:00:00,003XX W 64TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,HOSPITAL BUILDING/GROUNDS,False,False,722,7.0,20.0,68.0,11,NaN,NaN,2011,08/05/2017 03:50:08 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12358754,JE221969,2013-01-01 00:00:00,071XX S OAKLEY AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,832,8.0,18.0,66.0,11,NaN,NaN,2013,05/07/2021 04:49:27 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12359222,JE222451,2010-11-13 12:00:00,022XX S WHIPPLE ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER (SPECIFY),False,False,1033,10.0,12.0,30.0,11,NaN,NaN,2010,05/07/2021 04:49:27 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7977063,HT207697,2009-12-01 00:00:00,021XX W 19TH ST,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,False,True,1223,12.0,25.0,31.0,02,NaN,NaN,2009,05/12/2021 04:57:30 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_crime_ny = pd.read_csv('gdrive/MyDrive/Colab Notebooks/ny_sampled_data.csv', index_col=0)
df_crime_ny['Date'] = df_crime_ny['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ))
df_crime_ny = df_crime_ny.dropna(subset=['BORO_NM'])
df_crime_ny

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.



,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Date,Date64
CMPLNT_NUM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
242465164,05/09/2012,20:50:00,05/09/2012,21:00:00,24.0,05/09/2012,236,DANGEROUS WEAPONS,782.0,"WEAPONS, POSSESSION, ETC",COMPLETED,MISDEMEANOR,MANHATTAN,NaN,STREET,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,992820.0,231089.0,NaN,NaN,NaN,NaN,40.800966,-73.969047,"(40.800965968, -73.969047272)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E,2012-05-09 20:50:00,2012-05-09
593660503,02/20/2012,01:30:00,02/20/2012,02:00:00,32.0,02/20/2012,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,MANHATTAN,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1000693.0,234939.0,25-44,BLACK,M,NaN,40.811522,-73.940601,"(40.811522012, -73.940601181)",PATROL BORO MAN NORTH,NaN,25-44,BLACK,F,2012-02-20 01:30:00,2012-02-20
922264723,09/30/2012,16:00:00,10/02/2012,20:15:00,70.0,10/02/2012,341,PETIT LARCENY,321.0,"LARCENY,PETIT FROM AUTO",COMPLETED,MISDEMEANOR,BROOKLYN,FRONT OF,STREET,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,998399.0,167990.0,NaN,NaN,NaN,NaN,40.627767,-73.949029,"(40.627767171, -73.949028967)",PATROL BORO BKLYN SOUTH,NaN,25-44,WHITE,M,2012-09-30 16:00:00,2012-09-30
889259677,09/28/2012,09:30:00,09/28/2012,18:45:00,47.0,10/02/2012,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BRONX,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1026591.0,259192.0,25-44,BLACK HISPANIC,M,NaN,40.878003,-73.846892,"(40.878003133, -73.846891755)",PATROL BORO BRONX,NaN,18-24,BLACK HISPANIC,F,2012-09-28 09:30:00,2012-09-28
680485665,02/18/2012,15:15:00,02/18/2012,15:21:00,73.0,02/18/2012,106,FELONY ASSAULT,106.0,ASSAULT POLICE/PEACE OFFICER,ATTEMPTED,FELONY,BROOKLYN,NaN,STREET,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1007919.0,179199.0,NaN,NaN,NaN,NaN,40.658513,-73.914693,"(40.658513137, -73.914693387)",PATROL BORO BKLYN NORTH,NaN,NaN,UNKNOWN,E,2012-02-18 15:15:00,2012-02-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589512622,01/01/2010,17:17:00,01/01/2020,17:18:00,83.0,01/01/2020,351,CRIMINAL MISCHIEF & RELATED OF,254.0,"MISCHIEF, CRIMINAL 4, OF MOTOR",COMPLETED,MISDEMEANOR,BROOKLYN,FRONT OF,STREET,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1004923.0,191191.0,UNKNOWN,UNKNOWN,U,NaN,40.691436,-73.925455,"(40.69143597300007, -73.92545481499997)",PATROL BORO BKLYN NORTH,NaN,45-64,BLACK,M,2010-01-01 17:17:00,2010-01-01
358520029,03/01/2012,00:01:00,11/01/2012,23:59:00,43.0,01/03/2020,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,MISDEMEANOR,BRONX,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1020316.0,239179.0,25-44,BLACK,M,NaN,40.823101,-73.869690,"(40.82310129900002, -73.86969046099993)",PATROL BORO BRONX,NaN,25-44,WHITE,F,2012-03-01 00:01:00,2012-03-01
624411377,01/01/2010,01:00:00,01/01/2020,02:10:00,18.0,01/02/2020,109,GRAND LARCENY,416.0,"LARCENY,GRAND FROM NIGHT CLUB, UNATTENDED",COMPLETED,FELONY,MANHATTAN,INSIDE,BAR/NIGHT CLUB,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,987959.0,217674.0,UNKNOWN,UNKNOWN,U,NaN,40.764149,-73.986612,"(40.76414872600003, -73.98661223299997)",PATROL BORO MAN SOUTH,NaN,25-44,WHITE,F,2010-01-01 01:00:00,2010-01-01


In [ ]:
df_inc_ny = pd.DataFrame(data={'BOROUGH': ['MANHATTAN', 'BROOKLYN', 'BRONX', 'QUEENS', 'STATEN ISLAND'], 'INCOME': [85071, 56942, 37397, 64509, 79201]})
scaler = MinMaxScaler()
df_inc_ny['NORM INCOME'] = scaler.fit_transform(np.array(df_inc_ny['INCOME']).reshape(-1, 1))
df_inc_ny

,BOROUGH,INCOME,NORM INCOME
0,MANHATTAN,85071,1.000000
1,BROOKLYN,56942,0.409972
2,BRONX,37397,0.000000
3,QUEENS,64509,0.568696
4,STATEN ISLAND,79201,0.876872


In [ ]:
data_ny = df_crime_ny.set_index('BORO_NM').join(df_inc_ny.set_index('BOROUGH'), how='inner').reset_index().rename({'index':'BORO_NM'}, axis=1)
data_ny['year'] = data_ny['Date'].apply(lambda x: x.year)
data_ny['month'] = data_ny['Date'].apply(lambda x: x.month)
data_ny['day'] = data_ny['Date'].apply(lambda x: x.day)
data_ny['hour'] = data_ny['Date'].apply(lambda x: x.hour)
data_ny

,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Date,Date64,INCOME,NORM INCOME,year,month,day,hour
0,BRONX,09/28/2012,09:30:00,09/28/2012,18:45:00,47.0,10/02/2012,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1026591.0,259192.0,25-44,BLACK HISPANIC,M,NaN,40.878003,-73.846892,"(40.878003133, -73.846891755)",PATROL BORO BRONX,NaN,18-24,BLACK HISPANIC,F,2012-09-28 09:30:00,2012-09-28,37397,0.000000,2012,9,28,9
1,BRONX,10/13/2012,02:00:00,10/13/2012,15:00:00,50.0,10/14/2012,109,GRAND LARCENY,405.0,"LARCENY,GRAND BY THEFT OF CREDIT CARD",COMPLETED,FELONY,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1010515.0,262208.0,NaN,WHITE HISPANIC,M,NaN,40.886344,-73.905012,"(40.886343589, -73.90501245)",PATROL BORO BRONX,NaN,18-24,WHITE,M,2012-10-13 02:00:00,2012-10-13,37397,0.000000,2012,10,13,2
2,BRONX,02/04/2012,05:00:00,02/04/2012,05:22:00,52.0,02/04/2012,351,CRIMINAL MISCHIEF & RELATED OF,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1017542.0,255919.0,NaN,NaN,NaN,NaN,40.869059,-73.879630,"(40.869058532, -73.879630148)",PATROL BORO BRONX,NaN,25-44,WHITE HISPANIC,F,2012-02-04 05:00:00,2012-02-04,37397,0.000000,2012,2,4,5
3,BRONX,10/25/2012,20:25:00,10/25/2012,20:29:00,42.0,10/25/2012,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,NaN,785,1010919.0,239073.0,NaN,NaN,NaN,NaN,40.822844,-73.903643,"(40.822843728, -73.903643392)",PATROL BORO BRONX,NaN,NaN,UNKNOWN,E,2012-10-25 20:25:00,2012-10-25,37397,0.000000,2012,10,25,20
4,BRONX,06/03/2012,13:00:00,06/05/2012,14:00:00,41.0,07/03/2012,341,PETIT LARCENY,305.0,"LARCENY,PETIT BY CREDIT CARD U",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1011607.0,236142.0,NaN,NaN,NaN,NaN,40.814797,-73.901170,"(40.814796857, -73.901169522)",PATROL BORO BRONX,NaN,45-64,WHITE HISPANIC,F,2012-06-03 13:00:00,2012-06-03,37397,0.000000,2012,6,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2550223,STATEN ISLAND,07/15/2012,15:00:00,NaN,NaN,123.0,04/07/2020,233,SEX CRIMES,175.0,"SEXUAL ABUSE 3,2",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,914725.0,125837.0,45-64,WHITE,M,NaN,40.511804,-74.250035,"(40.51180433500008, -74.25003459199996)",PATROL BORO STATEN ISLAND,NaN,<18,WHITE,F,2012-07-15 15:00:00,2012-07-15,79201,0.876872,2012,7,15,15
2550224,STATEN ISLAND,03/24/2010,00:01:00,10/14/2010,00:01:00,122.0,03/20/2020,112,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",COMPLETED,FELONY,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,941533.0,138002.0,UNKNOWN,UNKNOWN,U,NaN,40.545364,-74.153702,"(40.54536449900007, -74.15370185199998)",PATROL BORO STATEN ISLAND,NaN,25-44,WHITE,M,2010-03-24 00:01:00,2010-03-24,79201,0.876872,2010,3,24,0
2550225,STATEN ISLAND,07/08/2009,20:00:00,07/08/2013,20:00:00,120.0,01/04/2020,116,SEX CRIMES,177.0,SEXUAL ABUSE,COMPLETED,FELONY,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,NaN,1292.0,962748.0,174174.0,18-24,BLACK,M,NaN,40.644726,-74.077483,"(40.64472613100002, -74.07748315899995)",PATROL BORO STATEN ISLAND,NaN,<18,BLACK HISPANIC,F,2009-07-08 20:00:00,2009-07-08,79201,0.876872,2009,7,8,20
2550226,STATEN ISLAND,05/01/2011,00:00:00,07/31/2013,23:59:00,120.0,01/03/2020,116,SEX CRIMES,168.0,SODOMY 1,COMPLETED,FELONY,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,962748.0

In [ ]:
df_inc = pd.read_csv('gdrive/MyDrive/Colab Notebooks/wealth.csv')
df_inc

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
...,...,...,...,...,...,...,...,...,...
73,74.0,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16.0
74,75.0,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30.0
75,76.0,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
76,77.0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [ ]:
data_coef = 1
data = df_crime[['Date', 'Community Areas', 'Primary Type']].sample(int(df_crime.shape[0]*data_coef))
data

,Date,Community Areas,Primary Type
ID,,,
7057358,2009-08-03 20:50:00,7.0,CRIMINAL TRESPASS
7222772,2009-11-11 07:00:00,65.0,BURGLARY
6413477,2008-08-06 09:00:00,9.0,OTHER OFFENSE
6116303,2008-03-04 20:53:43,26.0,NARCOTICS
6428039,2008-08-13 03:00:00,70.0,MOTOR VEHICLE THEFT
...,...,...,...
6205189,2008-04-19 23:40:00,14.0,BATTERY
6360671,2008-07-11 16:58:05,26.0,CRIMINAL DAMAGE
8025821,2011-04-19 18:00:00,45.0,OTHER OFFENSE


In [ ]:
data['year'] = data['Date'].apply(lambda x: x.year)
data['month'] = data['Date'].apply(lambda x: x.month)
data['day'] = data['Date'].apply(lambda x: x.day)
data['hour'] = data['Date'].apply(lambda x: x.hour)
data

,Date,Community Areas,Primary Type,year,month,day,hour
ID,,,,,,,
7057358,2009-08-03 20:50:00,7.0,CRIMINAL TRESPASS,2009,8,3,20
7222772,2009-11-11 07:00:00,65.0,BURGLARY,2009,11,11,7
6413477,2008-08-06 09:00:00,9.0,OTHER OFFENSE,2008,8,6,9
6116303,2008-03-04 20:53:43,26.0,NARCOTICS,2008,3,4,20
6428039,2008-08-13 03:00:00,70.0,MOTOR VEHICLE THEFT,2008,8,13,3
...,...,...,...,...,...,...,...
6205189,2008-04-19 23:40:00,14.0,BATTERY,2008,4,19,23
6360671,2008-07-11 16:58:05,26.0,CRIMINAL DAMAGE,2008,7,11,16
8025821,2011-04-19 18:00:00,45.0,OTHER OFFENSE,2011,4,19,18


In [ ]:
data = data.dropna()
data['Community Areas'] = data['Community Areas'].apply(lambda x: int(x))
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Community Areas,Primary Type,year,month,day,hour
ID,,,,,,,
7057358,2009-08-03 20:50:00,7,CRIMINAL TRESPASS,2009,8,3,20
7222772,2009-11-11 07:00:00,65,BURGLARY,2009,11,11,7
6413477,2008-08-06 09:00:00,9,OTHER OFFENSE,2008,8,6,9
6116303,2008-03-04 20:53:43,26,NARCOTICS,2008,3,4,20
6428039,2008-08-13 03:00:00,70,MOTOR VEHICLE THEFT,2008,8,13,3
...,...,...,...,...,...,...,...
6205189,2008-04-19 23:40:00,14,BATTERY,2008,4,19,23
6360671,2008-07-11 16:58:05,26,CRIMINAL DAMAGE,2008,7,11,16
8025821,2011-04-19 18:00:00,45,OTHER OFFENSE,2011,4,19,18


In [ ]:
data = data.set_index('Community Areas').join(df_inc.set_index('Community Area Number'), how='inner').reset_index().rename({'index':'Community Area Number'}, axis=1)
data

,Community Area Number,Date,Primary Type,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,2012-09-21 12:05:00,NARCOTICS,2012,9,21,12,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,1,2009-05-08 14:55:00,THEFT,2009,5,8,14,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2,1,2012-08-08 14:45:00,BURGLARY,2012,8,8,14,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
3,1,2012-12-04 08:00:00,THEFT,2012,12,4,8,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
4,1,2008-12-07 18:55:00,NARCOTICS,2008,12,7,18,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857897,77,2009-07-11 00:01:00,THEFT,2009,7,11,0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
1857898,77,2011-08-22 21:30:00,BURGLARY,2011,8,22,21,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
1857899,77,2010-02-19 00:05:00,MOTOR VEHICLE THEFT,2010,2,19,0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
1857900,77,2012-09-07 20:00:00,CRIMINAL DAMAGE,2012,9,7,20,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [ ]:
stats_border = 4
stats = data['Primary Type'].value_counts()
stats.head(stats_border)

THEFT              391178
BATTERY            327560
CRIMINAL DAMAGE    212790
NARCOTICS          204315
Name: Primary Type, dtype: int64

In [ ]:
data_sl = data.set_index('Primary Type', drop=True).join(
    pd.DataFrame(stats.head(stats_border)).drop('Primary Type', axis=1),
    how = 'inner').reset_index()
data_sl

,index,Community Area Number,Date,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,BATTERY,1,2009-07-03 11:30:00,2009,7,3,11,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,BATTERY,1,2008-04-07 18:10:00,2008,4,7,18,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2,BATTERY,1,2008-02-02 19:28:47,2008,2,2,19,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
3,BATTERY,1,2012-08-16 12:30:00,2012,8,16,12,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
4,BATTERY,1,2009-07-07 11:54:47,2009,7,7,11,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135838,THEFT,77,2011-01-16 20:00:00,2011,1,16,20,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
1135839,THEFT,77,2008-05-06 17:00:00,2008,5,6,17,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
1135840,THEFT,77,2008-08-12 21:15:00,2008,8,12,21,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
1135841,THEFT,77,2008-08-11 21:00:00,2008,8,11,21,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [ ]:
label_encoder = LabelEncoder()
data_sl['index'] = label_encoder.fit_transform(data_sl['index'])
data_sl['index'].value_counts()

3    391178
0    327560
1    212790
2    204315
Name: index, dtype: int64

In [ ]:
data_sl.drop(['index', 'Community Area Number', 'COMMUNITY AREA NAME', 'Date'],axis=1).columns

Index(['year', 'month', 'day', 'hour', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX'],
      dtype='object')

In [ ]:
# downsampling
classes = data_sl['index'].unique()
min_y = 10000000
for cls in classes:
  min_y = min(min_y, data_sl[data_sl['index'] == cls].shape[0])
res = []
for cls in classes:
  res.append(data_sl[data_sl['index'] == cls].sample(min_y))
data_sl = pd.concat(res, ignore_index=True)
data_sl

,index,Community Area Number,Date,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,0,76,2011-09-06 12:30:00,2011,9,6,12,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
1,0,20,2010-10-22 18:00:00,2010,10,22,18,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0
2,0,29,2008-04-23 14:20:00,2008,4,23,14,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0
3,0,32,2009-07-04 23:30:00,2009,7,4,23,Loop,1.5,14.7,5.7,3.1,13.5,65526,3.0
4,0,24,2010-04-01 22:00:00,2010,4,1,22,West Town,2.3,14.7,6.6,12.9,21.7,43198,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817255,3,29,2008-06-07 15:48:00,2008,6,7,15,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0
817256,3,68,2008-07-18 22:00:00,2008,7,18,22,Englewood,3.8,46.6,28.0,28.5,42.5,11888,94.0
817257,3,27,2011-09-01 11:36:00,2011,9,1,11,East Garfield Park,8.2,42.4,19.6,21.3,43.2,12961,83.0
817258,3,23,2012-01-14 03:30:00,2012,1,14,3,Humboldt park,14.8,33.9,17.3,35.4,38.0,13781,85.0


In [ ]:
data_sl.columns

Index(['index', 'Community Area Number', 'Date', 'year', 'month', 'day',
       'hour', 'COMMUNITY AREA NAME', 'PERCENT OF HOUSING CROWDED',
       'PERCENT HOUSEHOLDS BELOW POVERTY', 'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX'],
      dtype='object')

In [ ]:
scaler = MinMaxScaler()
data_sl['NORM INCOME'] = scaler.fit_transform(np.array(data_sl['PER CAPITA INCOME ']).reshape(-1, 1))
data_sl

,index,Community Area Number,Date,year,month,day,hour,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,NORM INCOME
0,0,76,2011-09-06 12:30:00,2011,9,6,12,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0,0.219056
1,0,20,2010-10-22 18:00:00,2010,10,22,18,Hermosa,6.9,20.5,13.1,41.6,36.4,15089,71.0,0.085599
2,0,29,2008-04-23 14:20:00,2008,4,23,14,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0,0.047634
3,0,32,2009-07-04 23:30:00,2009,7,4,23,Loop,1.5,14.7,5.7,3.1,13.5,65526,3.0,0.712395
4,0,24,2010-04-01 22:00:00,2010,4,1,22,West Town,2.3,14.7,6.6,12.9,21.7,43198,10.0,0.434918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817255,3,29,2008-06-07 15:48:00,2008,6,7,15,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0,0.047634
817256,3,68,2008-07-18 22:00:00,2008,7,18,22,Englewood,3.8,46.6,28.0,28.5,42.5,11888,94.0,0.045819
817257,3,27,2011-09-01 11:36:00,2011,9,1,11,East Garfield Park,8.2,42.4,19.6,21.3,43.2,12961,83.0,0.059154
817258,3,23,2012-01-14 03:30:00,2012,1,14,3,Humboldt park,14.8,33.9,17.3,35.4,38.0,13781,85.0,0.069344


In [ ]:
data_x_train, data_x_test, data_y_train, data_y_test = train_test_split(
        data_sl[['year', 'month', 'day', 'hour', 'NORM INCOME']],
        data_sl['index'],
        test_size = 0.20,
        random_state = 1551
)

In [ ]:
def find_best_params(classifier, parameters, data_x_train, data_y_train):
  pipeline = Pipeline([('sclr', MinMaxScaler()), ('clf', classifier)])
  grid = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2, scoring='f1_micro')
  grid.fit(data_x_train, data_y_train)
  return grid

In [ ]:
parameters = {'clf__learning_rate': [0.1],
 'clf__loss': ['auto'],
 'clf__max_iter': [70],
 'clf__max_leaf_nodes': [35],
 'clf__min_samples_leaf': [35]}

grid = find_best_params(HistGradientBoostingClassifier(), parameters, data_x_train, data_y_train)
grid.best_params_


Fitting 5 folds for each of 1 candidates, totalling 5 fits


{'clf__learning_rate': 0.1,
 'clf__loss': 'auto',
 'clf__max_iter': 70,
 'clf__max_leaf_nodes': 35,
 'clf__min_samples_leaf': 35}

In [ ]:
def launch(classifier, parameters, data_x_train, data_y_train, data_x_test, data_y_test, name):
  grid = find_best_params(classifier, parameters, data_x_train, data_y_train)
  estimator = grid.best_estimator_
  print("LAUNCH", name)
  print("TEST")
  prediction = estimator.predict(data_x_test)
  print(classification_report(data_y_test, prediction))
  print("TRAIN")
  prediction = estimator.predict(data_x_train)
  print(classification_report(data_y_train, prediction))
  print("-----")

In [ ]:
parameters = {'clf__criterion': ['entropy'],
 'clf__max_depth': [10],
 'clf__min_samples_leaf': [3],
 'clf__min_samples_split': [100],
 'clf__min_weight_fraction_leaf': [0.0],
 'clf__splitter': ['best']}
launch(DecisionTreeClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "DecisionTreeClassifier")

parameters = {'clf__criterion': ['entropy'],
 'clf__max_depth': [None],
 'clf__min_samples_leaf': [2],
 'clf__min_samples_split': [95],
 'clf__min_weight_fraction_leaf': [0.0],
 'clf__n_estimators': [150]}
launch(RandomForestClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "RandomForestClassifier")

parameters = {'clf__criterion': ['friedman_mse'],
 'clf__learning_rate': [0.4],
 'clf__loss': ['deviance'],
 'clf__max_depth': [3],
 'clf__min_samples_leaf': [3],
 'clf__min_samples_split': [2],
 'clf__n_estimators': [125]}
launch(GradientBoostingClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "GradientBoostingClassifier")

parameters = {'clf__learning_rate': [0.1],
 'clf__loss': ['auto'],
 'clf__max_iter': [70],
 'clf__max_leaf_nodes': [35],
 'clf__min_samples_leaf': [35]}
launch(HistGradientBoostingClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "HistGradientBoostingClassifier")

parameters = {'clf__activation': ['relu'],
 'clf__alpha': [0.0002],
 'clf__learning_rate': ['constant'],
 'clf__learning_rate_init':[0.002],
 'clf__max_iter': [250],
 'clf__solver': ['lbfgs', 'sgd']}
launch(MLPClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "MLPClassifier")

parameters = {'clf__C': [4.0],
 'clf__dual': [False],
 'clf__loss': ['squared_hinge'],
 'clf__max_iter': [1250],
 'clf__penalty': ['l1']}
launch(LinearSVC(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "LinearSVC")

parameters = {'clf__algorithm': ['SAMME.R'],
 'clf__base_estimator': [None],
 'clf__learning_rate': [1.5],
 'clf__n_estimators': [200]}
launch(AdaBoostClassifier(), parameters, data_x_train, data_y_train, data_x_test, data_y_test, "AdaBoostClassifier")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
LAUNCH AdaBoostClassifier
TEST
              precision    recall  f1-score   support

           0       0.34      0.23      0.28     40783
           1       0.35      0.29      0.32     40920
           2       0.42      0.58      0.49     40707
           3       0.44      0.49      0.46     41042

    accuracy                           0.40    163452
   macro avg       0.39      0.40      0.39    163452
weighted avg       0.39      0.40      0.39    163452

TRAIN
              precision    recall  f1-score   support

           0       0.35      0.23      0.28    163532
           1       0.35      0.29      0.32    163395
           2       0.42      0.58      0.49    163608
           3       0.44      0.49      0.46    163273

    accuracy                           0.40    653808
   macro avg       0.39      0.40      0.39    653808
weighted avg       0.39      0.40      0.39    653808

-----


In [ ]:
# testing on test set
estimator = grid.best_estimator_
prediction = estimator.predict(data_x_test)
print(classification_report(data_y_test, prediction))

In [ ]:
mp = {'DANGEROUS DRUGS':'NARCOTICS', 
 'THEFT-FRAUD':'THEFT',
 'GRAND LARCENY':'THEFT',
 'PETIT LARCENY':'THEFT',
 'CRIMINAL MISCHIEF & RELATED OF': 'CRIMINAL DAMAGE',
 'ASSAULT 3 & RELATED OFFENSES':'BATTERY',
 'FELONY ASSAULT':'BATTERY'}

data_ny['OFNS_DESC_MP'] = data_ny['OFNS_DESC'].apply(lambda x: mp[x] if x in mp else x)


In [ ]:
data_ny_clear = data_ny[(data_ny['OFNS_DESC_MP'] == 'THEFT') 
    | (data_ny['OFNS_DESC_MP'] == 'BATTERY') 
    | (data_ny['OFNS_DESC_MP'] == 'CRIMINAL DAMAGE')
    | (data_ny['OFNS_DESC_MP'] == 'NARCOTICS')]
data_ny_clear

,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Date,Date64,INCOME,NORM INCOME,year,month,day,hour,OFNS_DESC_MP
1,BRONX,10/13/2012,02:00:00,10/13/2012,15:00:00,50.0,10/14/2012,109,GRAND LARCENY,405.0,"LARCENY,GRAND BY THEFT OF CREDIT CARD",COMPLETED,FELONY,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1010515.0,262208.0,NaN,WHITE HISPANIC,M,NaN,40.886344,-73.905012,"(40.886343589, -73.90501245)",PATROL BORO BRONX,NaN,18-24,WHITE,M,2012-10-13 02:00:00,2012-10-13,37397,0.000000,2012,10,13,2,THEFT
2,BRONX,02/04/2012,05:00:00,02/04/2012,05:22:00,52.0,02/04/2012,351,CRIMINAL MISCHIEF & RELATED OF,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1017542.0,255919.0,NaN,NaN,NaN,NaN,40.869059,-73.879630,"(40.869058532, -73.879630148)",PATROL BORO BRONX,NaN,25-44,WHITE HISPANIC,F,2012-02-04 05:00:00,2012-02-04,37397,0.000000,2012,2,4,5,CRIMINAL DAMAGE
3,BRONX,10/25/2012,20:25:00,10/25/2012,20:29:00,42.0,10/25/2012,235,DANGEROUS DRUGS,567.0,"MARIJUANA, POSSESSION 4 & 5",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - PUBLIC HOUSING,N.Y. HOUSING POLICE,2.0,NaN,NaN,785,1010919.0,239073.0,NaN,NaN,NaN,NaN,40.822844,-73.903643,"(40.822843728, -73.903643392)",PATROL BORO BRONX,NaN,NaN,UNKNOWN,E,2012-10-25 20:25:00,2012-10-25,37397,0.000000,2012,10,25,20,NARCOTICS
4,BRONX,06/03/2012,13:00:00,06/05/2012,14:00:00,41.0,07/03/2012,341,PETIT LARCENY,305.0,"LARCENY,PETIT BY CREDIT CARD U",COMPLETED,MISDEMEANOR,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1011607.0,236142.0,NaN,NaN,NaN,NaN,40.814797,-73.901170,"(40.814796857, -73.901169522)",PATROL BORO BRONX,NaN,45-64,WHITE HISPANIC,F,2012-06-03 13:00:00,2012-06-03,37397,0.000000,2012,6,3,13,THEFT
5,BRONX,04/19/2012,15:00:00,NaN,NaN,45.0,04/19/2012,341,PETIT LARCENY,333.0,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,INSIDE,CHAIN STORE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,1030990.0,255310.0,NaN,NaN,NaN,NaN,40.867326,-73.831012,"(40.867326052, -73.831011607)",PATROL BORO BRONX,NaN,NaN,UNKNOWN,D,2012-04-19 15:00:00,2012-04-19,37397,0.000000,2012,4,19,15,THEFT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2550219,STATEN ISLAND,01/01/2013,16:00:00,12/31/2016,15:00:00,121.0,09/03/2020,112,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",COMPLETED,FELONY,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,938149.0,171781.0,NaN,NaN,NaN,NaN,40.638064,-74.166108,"(40.63806398200006, -74.16610830199994)",PATROL BORO STATEN ISLAND,NaN,25-44,BLACK,F,2013-01-01 16:00:00,2013-01-01,79201,0.876872,2013,1,1,16,THEFT
2550220,STATEN ISLAND,01/01/2013,00:01:00,NaN,NaN,123.0,08/18/2020,109,GRAND LARCENY,402.0,"LARCENY,GRAND BY FALSE PROMISE-NOT IN PERSON C...",COMPLETED,FELONY,INSIDE,RESIDENCE-HOUSE,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,925557.0,140795.0,UNKNOWN,WHITE,F,NaN,40.552939,-74.211209,"(40.55293940000007, -74.21120927799996)",PATROL BORO STATEN ISLAND,NaN,45-64,WHITE,F,2013-01-01 00:01:00,2013-01-01,79201,0.876872,2013,1,1,0,THEFT
2550221,STATEN ISLAND,04/01/2010,00:00:00,04/15/2020,00:00:00,123.0,05/19/2020,351,CRIMINAL MISCHIEF & RELATED OF,258.0,"CRIMINAL MISCHIEF 4TH, GRAFFIT",COMPLETED,MISDEMEANOR,REAR OF,STORAGE FACILITY,N.Y. POLICE DEPT,0.0,NaN,NaN,NaN,918418.0,130863.0,NaN,NaN,NaN,NaN,40.525628,-74.236802,"(40.525627900000075, -74.23680236299998)",PATROL BORO STATEN ISLAND,NaN,UNKNOWN,UNKNOWN,D,2010-04-01 00:00:00,2010-04-01,79201,0.876872,2010,4,1,0,CRIMINAL DAMAGE
2550222,STATEN ISLAND,06/11/2012,00:01:00,06/25/2018,10:03:00,121.0,05/27

In [ ]:
estimator = grid.best_estimator_
prediction = estimator.predict(data_ny_clear[['year', 'month', 'day', 'hour', 'NORM INCOME']])
original = label_encoder.transform(data_ny_clear['OFNS_DESC_MP'])

In [ ]:
pd.DataFrame({'prediction':label_encoder.inverse_transform(prediction), 'original':data_ny_clear['OFNS_DESC_MP'], 'code':prediction})

,prediction,original,code
1,CRIMINAL DAMAGE,THEFT,1
2,CRIMINAL DAMAGE,CRIMINAL DAMAGE,1
3,CRIMINAL DAMAGE,NARCOTICS,1
4,THEFT,THEFT,3
5,THEFT,THEFT,3
...,...,...,...
2550219,BATTERY,THEFT,0
2550220,THEFT,THEFT,3
2550221,THEFT,CRIMINAL DAMAGE,3
2550222,BATTERY,THEFT,0


In [ ]:
print(classification_report(original, prediction))


              precision    recall  f1-score   support

           0       0.30      0.26      0.28    346604
           1       0.20      0.26      0.23    256394
           2       0.16      0.20      0.18    190113
           3       0.57      0.49      0.53    640378

    accuracy                           0.36   1433489
   macro avg       0.31      0.30      0.30   1433489
weighted avg       0.38      0.36      0.37   1433489



In [ ]:
# testing on train set
prediction = estimator.predict(data_x_train)
print(classification_report(data_y_train, prediction))

              precision    recall  f1-score   support

           0       0.37      0.26      0.31    163532
           1       0.38      0.32      0.35    163395
           2       0.44      0.60      0.51    163608
           3       0.45      0.49      0.47    163273

    accuracy                           0.42    653808
   macro avg       0.41      0.42      0.41    653808
weighted avg       0.41      0.42      0.41    653808



In [ ]:
kmeans = KMeans(n_clusters=4).fit(data_sl[['year', 'month', 'day', 'hour', 'NORM INCOME', 'Community Area Number']])
kmeans.labels_

array([3, 1, 0, ..., 2, 2, 2], dtype=int32)

In [ ]:
to_draw = data_sl[(data_sl['year'] == 2011) & (data_sl['month'] == 2)]

px.scatter_3d(pd.concat(
     [
     to_draw.reset_index(drop=True),
     pd.DataFrame(kmeans.predict(to_draw[['year', 'month', 'day', 'hour', 'NORM INCOME', 'Community Area Number']]))
     ],
     axis = 1)
, x='day', y='hour', z='NORM INCOME', color=0)

In [ ]:
pdc = pd.concat(
      [
      to_draw.reset_index(drop=True),
      pd.DataFrame(kmeans.predict(to_draw[['year', 'month', 'day', 'hour', 'NORM INCOME', 'Community Area Number']]))
      ], 
      axis = 1)

px.scatter_3d(pdc, x='day', y='hour', z='NORM INCOME', color='index')

In [ ]:
estimator = grid.best_estimator_
pdc = pd.concat([to_draw.reset_index(drop=True),pd.DataFrame(estimator.predict(to_draw[['year', 'month', 'day', 'hour', 'NORM INCOME']]))], axis = 1)
px.scatter_3d(pdc[(pdc[0]==2) | (pdc['index'] == 2)], x='day', y='hour', z='NORM INCOME', color=0)

In [ ]:
px.scatter_3d(pd.concat([to_draw.reset_index(drop=True),pd.DataFrame(estimator.predict(to_draw[['year', 'month', 'day', 'hour', 'NORM INCOME']]))], axis = 1)
, x='day', y='hour', z='Community Area Number', color=0)

In [ ]:
px.scatter_3d(to_draw
, x='day', y='hour', z='Community Area Number', color='index')

In [ ]:
px.scatter_3d(pd.concat([to_draw.reset_index(drop=True),pd.DataFrame(kmeans.predict(to_draw[['year', 'month', 'day', 'hour', 'NORM INCOME', 'Community Area Number']]))], axis = 1)
, x='day', y='hour', z='Community Area Number', color=0)